<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/rust01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

1. Colab で実行しながら読まれることを想定している。
1. Rust の勉強
1. 資料は本家のテュートリアル https://rust-cli.github.io/book/index.html

# Rust のインストール





In [ ]:
# Rust をインストールする。
%%capture
!apt install rustc --fix-missing

通常の curl を使ったインストールは使えない。

In [ ]:
# 参考

# !curl https://sh.rustup.rs -sSf | sh
#=> sh: 121: cannot open /dev/tty: No such device or address

In [ ]:
# インストールの確認
!rustc --version

/bin/bash: rustc: command not found


In [ ]:
# 環境をつくるコマンドの cargo も入っている
!cargo --version

/bin/bash: cargo: command not found


In [ ]:
%env USER=root
!cargo init

env: USER=root
     Created binary (application) package


以上で、ここでmain.rsをビルドして、コンパイルすることができるようになる。

In [ ]:
# プログラムを書く
%%writefile main.rs
fn main() {
    println!("Hello, world!");
}

Overwriting main.rs


In [ ]:
# コンパイル
!cargo build

   Compiling content v0.1.0 (/content)
    Finished dev [unoptimized + debuginfo] target(s) in 0.40s


In [ ]:
# コンパイルして実行
!cargo run 

   Compiling content v0.1.0 (/content)
error[E0423]: expected function, found macro `println`
 --> main.rs:2:5
  |
2 |     println("Hello, world!");
  |     ^^^^^^^ help: use `!` to invoke the macro: `println!`

error: aborting due to previous error

For more information about this error, try `rustc --explain E0423`.
error: could not compile `content`.

To learn more, run the command again with --verbose.


# テュートリアルより



以上で Colab で Rust チュートリアルのノートをとる準備が整ったが、チュートリアル自体は Colab 用ではないので次のような記述になっている。

まず、`cargo new grrs` (grrsは今回作るプログラム名)、でディレクトリーを作る。 grrs の中にいろいろなファイルができる。 なかでも `Cargo.toml` にプロジェクトのメタデータが書かれていて、使われるライブラリーの依存、バージョンコントロールが行われる。 cargo run すると、main.rs がコンパイルされ、実行される。

In [ ]:
# ざっと次のような流れになる
%%script false
$ cargo new grrs
     Created binary (application) `grrs` package
$ cd grrs/
$ cargo run
   Compiling grrs v0.1.0 (/Users/pascal/code/grrs)
    Finished dev [unoptimized + debuginfo] target(s) in 0.70s
     Running `target/debug/grrs`
Hello, world!

セルマジックの %%script false が書かれるとそれ以降の行は実行されない。

全体をコメントアウトしたような使い方ができる。

## 最初の課題

In [ ]:
# テュートリアルより最初の課題
# 次のような動作をするgrepもどき(grrs)をつくる。

# $ cat test.txt
# foo: 10
# bar: 20
# baz: 30
# $ grrs foo test.txt
# foo: 10
# $ grrs --help
# [some help text explaining the available options]


## コマンドラインをパースする

典型的なコマンドラインツールはつぎのような形になる。


In [ ]:
%%script false
$ grrs foobar test.txt

この例では、grrs というプログラムで、test.txt の中の foobar という文字列を含む行をプリントアウトすることを想定している。

プログラム名のあとの文字列はオペレーティングシステムの内部的には大雑把にいうとスペースで区切られている文字列のリストとして処理されている。

コマンドライン引数と呼ばれる。 --this というような書き方でフラグをしていするのにも使われる。




## コマンドライン引数の取得

標準ライブラリーに関数 
std::env::args() 
があり、戻り値はイテレーターで index 0 にはプログラム自身、今回の例では grrs が入っている。 

引数を含めた全体はつぎのようなプログラムで取得する。




In [ ]:
%%script false
fn main() {
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");
}

Rust ではデータを中心にプログラムを組み立てるのが通例である。 それにしたがって、次のように書く。

In [ ]:
%%script false

struct Cli {
    pattern: String,
    path: std::path::PathBuf,
}

fn main (){
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");

    let args = Cli {
        pattern: pattern,
        path: std::path::PathBuf::from(path),
};



let pattern = std::env::args().nth(1).expect("no pattern given");
let path = std::env::args().nth(2).expect("no path given");
let args = Cli {
    pattern: pattern,
    path: std::path::PathBuf::from(path),
};
This works, but it’s not very convenient. How would you deal with the requirement to support --pattern="foo" or --pattern "foo"? How would you implement --help?


## StructOpt によるコマンドライン引数の取得

上のプログラムでもコマンドライン引数は取得できるのだが、あまり便利ではない。 フラグやオプション、ヘルプを付け加えたいというときにどうするか。 ライブラリーを使うのが便利で、コマンドライン引数をパースするのによく使われるのは clap というライブラリーである。

今回使う structopt ライブラリーも clap をベースにしていて、struct の定義に使える。

ライブラリーを import するには、Cargo.toml の [dependencies] セクションに
```
structopt = "0.3.13"
```
を加える。

In [ ]:
%%writefile Cargo.toml
[package]
name = "content"
version = "0.1.0"
authors = ["root"]
edition = "2018"

# See more keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html

[dependencies]
structopt = "0.3.13"

[[bin]]
name = "content"
path = "main.rs"

Overwriting Cargo.toml


これで次のようにプログラムで StructOpt が使える。

ついでにドキュメントも加えよう。



In [ ]:
%%writefile main.rs
use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

fn main (){
    let args = Cli::from_args();
}

Overwriting main.rs


Clap は各フィールドに入るべき値を管理していて、

```    
let args = Cli::from_args();

```
が失敗すると、ヘルプメッセージを出して、終了する。



In [ ]:
# あるべき引数なしに実行するとつぎのようになる
%%script false
$ cargo run
    Finished dev [unoptimized + debuginfo] target(s) in 10.16s
     Running `target/debug/grrs`
error: The following required arguments were not provided:
    <pattern>
    <path>

USAGE:
    grrs <pattern> <path>

For more information try --help

In [ ]:
# cargo run に引数を付けて実行したい時は -- のあとに書く

%%script false

$ cargo run -- some-pattern some-file
    Finished dev [unoptimized + debuginfo] target(s) in 0.11s
     Running `target/debug/grrs some-pattern some-file`

実験: やってみよう。

In [ ]:
# 実験: やってみる
!cargo run

   Compiling content v0.1.0 (/content)
  --> main.rs:14:9
   |
14 |     let args = Cli::from_args();
   |         ^^^^ help: consider prefixing with an underscore: `_args`
   |
   = note: `#[warn(unused_variables)]` on by default

 --> main.rs:7:5
  |
7 |     pattern: String,
  |     ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(dead_code)]` on by default

  --> main.rs:10:5
   |
10 |     path: std::path::PathBuf,
   |     ^^^^^^^^^^^^^^^^^^^^^^^^

    Finished dev [unoptimized + debuginfo] target(s) in 0.99s
     Running `target/debug/content`
error: The following required arguments were not provided:
    <pattern>
    <path>

USAGE:
    content <pattern> <path>

For more information try --help


In [ ]:
# 実験 引数を付けてみる
!cargo run -- some-pattern some-file

  --> main.rs:14:9
   |
14 |     let args = Cli::from_args();
   |         ^^^^ help: consider prefixing with an underscore: `_args`
   |
   = note: `#[warn(unused_variables)]` on by default

 --> main.rs:7:5
  |
7 |     pattern: String,
  |     ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(dead_code)]` on by default

  --> main.rs:10:5
   |
10 |     path: std::path::PathBuf,
   |     ^^^^^^^^^^^^^^^^^^^^^^^^

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/content some-pattern some-file`


## 課題 pattern と path を出力する

In [ ]:
%%writefile main.rs
use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

fn main (){
    let args = Cli::from_args();
    println!("{} => {:?}", args.pattern, args.path)
}

Overwriting main.rs


In [ ]:
# 実験
!cargo run -- some-pattern some-file

   Compiling content v0.1.0 (/content)
    Finished dev [unoptimized + debuginfo] target(s) in 1.04s
     Running `target/debug/content some-pattern some-file`
some-pattern => "some-file"


## grrs の最初の実装


In [ ]:
%%writefile main.rs
use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

fn main (){
    let args = Cli::from_args();
    let content = std::fs::read_to_string(&args.path).expect("could not read file");

    for line in content.lines() {
        if line.contains(&args.pattern) {
            println!("{}", line);
        }
    }
}

Overwriting main.rs


In [ ]:
!cargo run -- main main.rs

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/content main main.rs`
fn main (){


上の実行結果が、
```
fn main (){
```
であれば成功!!!!

In [ ]:
# 実験
!cargo run -- path main.rs

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/content path main.rs`
    /// The path to the file to read
    path: std::path::PathBuf,
    let content = std::fs::read_to_string(&args.path).expect("could not read file");


## 課題 ファイルを一度に全部読み込まない方法

巨大なファイルだった場合、
```
let content = std::fs::read_to_string(&args.path).expect("could not read file");
```
`read_to_string()`の代わりに`BufReader`を使う方法がある。



## エラー出力の改善



## Results

関数 read_to_string のような関数は戻り値は文字列ではなく、文字列かもしくはなんらかのエラーという Result というタイプの戻り値になる。 エラーは今回の場合は std::io::Error になる。 Result は列挙型 enum なので、結果は match が使える。



In [ ]:
%%script false
let result = std::fs::read_to_string("test.txt");
match result {
    Ok(content) => { println!("File content: {}", content); }
    Err(error) => { println!("Oh noes: {}", error); }
}

## アンラッピング

match によって得られた文字列 content は次のような方法で変数に格納できる。


In [ ]:
%%script false
let result = std::fs::read_to_string("test.txt");
let content = match result {
    Ok(content) => { content },
    Err(error) => { panic!("Can't deal with {}, just exit here", error); }
};
println!("file content: {}", content);

# いまここ


We can use the String in content after the match block. If result were an error, the String wouldn’t exist. But since the program would exit before it ever reached a point where we use content, it’s fine.

This may seem drastic, but it’s very convenient. If your program needs to read that file and can’t do anything if the file doesn’t exist, exiting is a valid strategy. There’s even a shortcut method on Results, called unwrap:




```
let content = std::fs::read_to_string("test.txt").unwrap();
```



# No need to panic

Of course, aborting the program is not the only way to deal with errors. Instead of the panic!, we can also easily write return:




```
let result = std::fs::read_to_string("test.txt");
let _content = match result {
    Ok(content) => { content },
    Err(error) => { return Err(error.into()); }
};
```



This, however changes the return type our function needs. Indeed, there was something hidden in our examples all this time: The function signature this code lives in. And in this last example with return, it becomes important. Here’s the full example:




```
fn main() -> Result<(), Box<dyn std::error::Error>> {
    let result = std::fs::read_to_string("test.txt");
    let content = match result {
        Ok(content) => { content },
        Err(error) => { return Err(error.into()); }
    };
    println!("file content: {}", content);
    Ok(())
}
```



Our return type is a Result! This is why we can write return Err(error); in the second match arm. See how there is an Ok(()) at the bottom? It’s the default return value of the function and means “Result is okay, and has no content”.

Aside: Why is this not written as return Ok(());? It easily could be – this is totally valid as well. The last expression of any block in Rust is its return value, and it is customary to omit needless returns.



# Question Mark

Just like calling .unwrap() is a shortcut for the match with panic! in the error arm, we have another shortcut for the match that returns in the error arm: ?.

That’s right, a question mark. You can append this operator to a value of type Result, and Rust will internally expand this to something very similar to the match we just wrote.

Give it a try:



```

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let content = std::fs::read_to_string("test.txt")?;
    println!("file content: {}", content);
    Ok(())
}
```


Very concise!

Aside: There are a few more things happening here that are not required to understand to work with this. For example, the error type in our main function is Box<dyn std::error::Error>. But we’ve seen above that read_to_string returns a std::io::Error. This works because ? expands to code that converts error types.

Box<dyn std::error::Error> is also an interesting type. It’s a Box that can contain any type that implements the standard Error trait. This means that basically all errors can be put into this box, so we can use ? on all of the usual functions that return Results.

## Providing Context

The errors you get when using ? in your main function are okay, but they are not great. For example: When you run std::fs::read_to_string("test.txt")? but the file test.txt doesn’t exist, you get this output:


Error: Os { code: 2, kind: NotFound, message: "No such file or directory" }
In cases where your code doesn’t literally contain the file name, it would be very hard to tell which file was NotFound. There are multiple ways to deal with this.

For example, we can create our own error type, and then use that to build a custom error message:



```
#[derive(Debug)]
struct CustomError(String);

fn main() -> Result<(), CustomError> {
    let path = "test.txt";
    let content = std::fs::read_to_string(path)
        .map_err(|err| CustomError(format!("Error reading `{}`: {}", path, err)))?;
    println!("file content: {}", content);
    Ok(())
}

```



Now, running this we’ll get our custom error message:

```
Error: CustomError("Error reading `test.txt`: No such file or directory (os error 2)")
Not very pretty, but we can easily adapt the debug output for our type later on.
```
This pattern is in fact very common. It has one problem, though: We don’t store the original error, only its string representation. The often used anyhow library has a neat solution for that: Similar to our CustomError type, its Context trait can be used to add a description. Additionally, it also keeps the original error, so we get a “chain” of error messages pointing out the root cause.

Let’s first import the anyhow crate by adding anyhow = "1.0" to the [dependencies] section of our Cargo.toml file.

The full example will then look like this:



```
use anyhow::{Context, Result};

fn main() -> Result<()> {
    let path = "test.txt";
    let content = std::fs::read_to_string(path)
        .with_context(|| format!("could not read file `{}`", path))?;
    println!("file content: {}", content);
    Ok(())
}
This will print an error:


Error: could not read file `test.txt`

Caused by:
    No such file or directory (os error 2)
```